In [ ]:
import cv2
import os
import numpy as np
import pandas as pd

from keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras.models import Sequential,load_model
from keras.optimizers import Adam


from keras.callbacks import ModelCheckpoint, TensorBoard

SHAPE = (160, 320, 3)

# Yeni Bölüm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Yeni Bölüm

# Zenginleştirme sırasında kullanılacak fonksiyonlar

In [ ]:
def crop_resize(image, result_shape = (160, 320)):
    preprocessed_image = image[180::]
    preprocessed_image = cv2.resize(preprocessed_image, result_shape, cv2.INTER_AREA)
    return preprocessed_image

def random_brightness(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    ratio = 1.0 + 0.4 * (np.random.rand() - 0.5)
    hsv[:,:,2] =  hsv[:,:,2] * ratio
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)


def random_shadow(image):
    h, w = image.shape[:-1]
    x1, y1 = w * np.random.rand(), 0
    x2, y2 = w * np.random.rand(), h
    xm, ym = np.mgrid[0:h, 0:w]

    mask = np.zeros_like(image[:, :, 1])
    mask[(ym - y1) * (x2 - x1) - (y2 - y1) * (xm - x1) > 0] = 1

    cond = mask == np.random.randint(2)
    s_ratio = np.random.uniform(low=0.2, high=0.5)

    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    hls[:, :, 1][cond] = hls[:, :, 1][cond] * s_ratio
    return cv2.cvtColor(hls, cv2.COLOR_HLS2RGB)


def random_flip(image, angle):
    if np.random.rand() < 0.5:
        image = cv2.flip(image, 1)
        angle = -angle
    return image, angle


def random_salt_and_pepper(image):
    if(np.random.rand() < .10):
        return image
    row,col,ch = image.shape
    s_vs_p = 0.5
    amount = 0.02
    out = np.copy(image)
    # Salt mode
    num_salt = np.ceil(amount * image.size * s_vs_p)
    coords = [np.random.randint(0, i - 1, int(num_salt))
            for i in image.shape]
    out[coords] = 1

    # Pepper mode
    num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
    coords = [np.random.randint(0, i - 1, int(num_pepper))
            for i in image.shape]
    out[coords] = 0
    return out

# Zenginleştirme (image augmentation)

In [ ]:
def data_generator(directory, batch_size, image_pre_path = "."):
    

    df = pd.read_csv(directory)
    samples_len = len(df)
    
    while True:
        train_x = []
        train_y = []
        for _ in range(batch_size):
            sample_index = np.random.randint(samples_len)

            image_file_name = df.loc[sample_index, "image_name"]
            #action_x =  df.loc[sample_index, "x"]
            action_y =  df.loc[sample_index, "y"]

            image = cv2.imread(f"{image_pre_path}/{image_file_name}")
            
            image = crop_resize(image)
            image = random_brightness(image)
            image = random_shadow(image)
            image, action_y = random_flip(image, action_y)
            image = random_salt_and_pepper(image)

            train_x.append(image)
            train_y.append(action_y)

        train_x = np.array(train_x)
        train_y = np.array(train_y)
        yield train_x.reshape((-1, SHAPE[0], SHAPE[1], SHAPE[2])), train_y.reshape((-1, 1))


# Model oluşturma

In [ ]:
def build_model():
    model=Sequential()
    model.add(Lambda(lambda x: x/127.5-1.0,input_shape=(160,320,3)))
    #ELU(Exponential linear unit) function takes care of the Vanishing gradient problem.
    model.add(Conv2D(24,(5,5),activation="elu",strides=(2,2)))
    model.add(Conv2D(36,(5,5),activation="elu",strides=(2,2)))
    model.add(Conv2D(48,(5,5),activation="elu",strides=(2,2)))
    model.add(Conv2D(64,(5,5),activation="elu"))
    model.add(Conv2D(64,(5,5),activation="elu"))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(100,activation="elu"))
    model.add(Dense(50,activation="elu"))
    model.add(Dense(10,activation="elu"))
    model.add(Dense(1))

    model.summary()
    return model

In [ ]:
model = build_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              (None, 160, 320, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 78, 158, 24)       1824      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 37, 77, 36)        21636     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 37, 48)        43248     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 33, 64)        76864     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 9, 29, 64)         102464    
_________________________________________________________________
dropout (Dropout)            (None, 9, 29, 64)         0

# Eğitim

In [ ]:
def train(model, csv_path):
	#path verinin kaydedildiği csv yi gösterecek
	model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.0001),metrics=["accuracy"])
   
    #####
    #path olarak yazılan yere uygun adresi ver
    #####
	checkpoint = ModelCheckpoint("path4/bin_v{epoch:02d}.hdf5", verbose=1) #val_acc yok ondan val_loss

	"""
	fit_generator(object, generator, steps_per_epoch, epochs = 1,
		verbose = getOption("keras.fit_verbose", default = 1),
		callbacks = NULL, view_metrics = getOption("keras.view_metrics",
		default = "auto"), validation_data = NULL, validation_steps = NULL,
		class_weight = NULL, max_queue_size = 10, workers = 1,
		initial_epoch = 0)
	"""
    #fotolar da images klasörünün içinde olacak şekide yazdım ama 
    #fotolar drive'a yüklenirse (ki daha kolay olur) google drive colab'a bağlanmalı ve
    #image_pre_path argumanı driveda fotoların olduğu yol olmalı

    #steps_per_epoch * 32 data oluşturulacak her epoch da
	model.fit_generator(data_generator(csv_path, 32, image_pre_path = "/content/drive/MyDrive/duckie-town/images"),
						steps_per_epoch = 10000,
						epochs = 10,
						validation_data=data_generator(csv_path, 100, image_pre_path = "/content/drive/MyDrive/duckie-town/images"),
						validation_steps = 5,
						callbacks=[TensorBoard(log_dir='/content/drive/MyDrive/models (5)/', histogram_freq=0, write_graph=False), checkpoint ])
 
    #####
    #path olarak yazılan yere uygun adresi ver
    #####
	model.save("/content/drive/MyDrive/models (5)/binary.hdf5")

In [ ]:
train(model, "drive/MyDrive/duckie-town/data/all.csv")

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:54: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


Epoch 1/10
2000/2000 [==============================] - 1798s 898ms/step - loss: 0.1592 - accuracy: 0.5054 - val_loss: 0.0532 - val_accuracy: 0.5280

Epoch 00001: saving model to path4/bin_v01.hdf5
Epoch 2/10
2000/2000 [==============================] - 705s 353ms/step - loss: 0.0567 - accuracy: 0.5161 - val_loss: 0.0430 - val_accuracy: 0.4920

Epoch 00002: saving model to path4/bin_v02.hdf5
Epoch 3/10
2000/2000 [==============================] - 707s 354ms/step - loss: 0.0387 - accuracy: 0.5177 - val_loss: 0.0298 - val_accuracy: 0.5240

Epoch 00003: saving model to path4/bin_v03.hdf5
Epoch 4/10
2000/2000 [==============================] - 710s 355ms/step - loss: 0.0307 - accuracy: 0.5197 - val_loss: 0.0245 - val_accuracy: 0.5180

Epoch 00004: saving model to path4/bin_v04.hdf5
Epoch 5/10
2000/2000 [==============================] - 707s 354ms/step - loss: 0.0244 - accuracy: 0.5162 - val_loss: 0.0142 - val_accuracy: 0.5220

Epoch 00005: saving model to path4/bin_v05.hdf5
Epoch 6/10
200

In [ ]:

x = load_model("path4/bin_v04.hdf5")

x.save_weights("path4/bin_weights4.h5")



In [ ]:
model = load_model("drive/MyDrive/models (4)/binary.hdf5")

In [ ]:
train(model, "drive/MyDrive/duckie-town/data/all.csv")

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:54: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


Epoch 1/10
10000/10000 [==============================] - 4891s 488ms/step - loss: 0.0084 - accuracy: 0.5210 - val_loss: 0.0036 - val_accuracy: 0.5180

Epoch 00001: saving model to path4/bin_v01.hdf5
Epoch 2/10
10000/10000 [==============================] - 3453s 345ms/step - loss: 0.0050 - accuracy: 0.5230 - val_loss: 0.0024 - val_accuracy: 0.5420

Epoch 00002: saving model to path4/bin_v02.hdf5
Epoch 3/10
10000/10000 [==============================] - 3447s 345ms/step - loss: 0.0034 - accuracy: 0.5233 - val_loss: 0.0012 - val_accuracy: 0.5340

Epoch 00003: saving model to path4/bin_v03.hdf5
Epoch 4/10
10000/10000 [==============================] - 3464s 346ms/step - loss: 0.0024 - accuracy: 0.5213 - val_loss: 0.0012 - val_accuracy: 0.5220

Epoch 00004: saving model to path4/bin_v04.hdf5
Epoch 5/10
 5377/10000 [===============>..............] - ETA: 27:04 - loss: 0.0019 - accuracy: 0.5217

KeyboardInterrupt: ignored